# 第5回講義 宿題

## 課題

今Lessonで学んだことを元に、MNISTのファッション版 (Fashion MNIST、クラス数10) を多層パーセプトロンによって分類してみましょう。

Fashion MNISTの詳細については以下のリンクを参考にしてください。

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy: 85%

### ルール

- **下のセルで指定されている`x_train`、`y_train`以外の学習データは使わないでください。**
- MLPの実装には`tf`の低レベルAPIのみを用いてください。具体的に以下のモジュールは使用しないでください。
```python
tf.app,
tf.compat,
tf.contrib,
tf.estimator,
tf.gfile,
tf.graph_util,
tf.image,
tf.initializers,
tf.keras,
tf.layers,
tf.logging,
tf.losses,
tf.metrics,
tf.python_io,
tf.resource_loader,
tf.saved_model,
tf.sets,
tf.summary,
tf.sysconfig,
tf.test,
tf.train
```

### 提出方法

### 提出方法
- 2つのファイルを提出していただきます。
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し、**Homeworkタブから`chap05`を選択して**提出してください。
    2. それに対応するpythonのコードを`submission_code.py`として保存し、**Homeworkタブから`chap05 (code)`を選択して**提出してください。
      - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください（writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください）。
      
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。

### 評価方法

- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します.
- 毎日夜24時にテストデータの一部に対する精度でLeader Boardを更新します.
- 締切日の夜24時にテストデータ全体に対する精度でLeader Boardを更新します. これを最終的な評価とします.

### データの読み込み
- この部分は修正しないでください。

In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

try:
    del [
        tf.app,
        tf.compat,
        tf.contrib,
        tf.estimator,
        tf.gfile,
        tf.graph_util,
        tf.image,
        tf.initializers,
        tf.keras,
        tf.layers,
        tf.logging,
        tf.losses,
        tf.metrics,
        tf.python_io,
        tf.resource_loader,
        tf.saved_model,
        tf.sets,
        tf.summary,
        tf.sysconfig,
        tf.test,
        tf.train
    ]
except AttributeError:
    print('Unrequired modules are already deleted (Skipped).')

def load_fashionmnist():
    # 学習データ
    x_train = np.load('/root/userspace/public/chap05/data/x_train.npy')
    y_train = np.load('/root/userspace/public/chap05/data/y_train.npy')
    
    # テストデータ
    x_test = np.load('/root/userspace/public/chap05/data/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

In [2]:
x_train, y_train, x_test = load_fashionmnist()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(60000, 784)
(60000, 10)
(10000, 784)


### 多層パーセプトロンの実装

In [60]:
# %%writefile /root/userspace/chap05/materials/submission_code.py

import math
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#乱数のseedの前にもってきた
tf.reset_default_graph() # グラフのリセット

#乱数について追加
random_seed = 34
np.random.seed(random_seed)
tf.set_random_seed(random_seed) 

x_train, y_train, x_test = load_fashionmnist()
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=1000)

x = tf.placeholder(tf.float32, [None, 784])
t = tf.placeholder(tf.float32, [None, 10])
#勝手に追加
is_training = tf.placeholder(tf.bool) # 訓練時orテスト時

class Dense:
    # WRITE ME
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        self.W = tf.Variable(tf.random_uniform(shape=(in_dim, out_dim), minval=-0.08, maxval=0.08), name='W')
        self.b = tf.Variable(tf.zeros(out_dim), name='b')
        self.function = function
        
        self.params = [self.W, self.b]
    
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
    
class Dropout: ##勝手に追加
    def __init__(self, dropout_keep_prob=1.0):
        self.dropout_keep_prob = dropout_keep_prob
        self.params = []
    
    def __call__(self, x):
        # 訓練時のみdropoutを適用
        return tf.cond(
            pred=is_training,
            true_fn=lambda: tf.nn.dropout(x, keep_prob=self.dropout_keep_prob),
            false_fn=lambda: x
        )

def sgd(cost, params, eta=0.1): #epsになってたので、etaに変更。Adagradとかも試してみるといいかも
    # WRITE ME
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        updates.append(param.assign_sub(eta * grad))
    return updates

def adagrad(cost, params, eta=0.01, eps=1e-7):
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        G = tf.Variable(tf.zeros_like(param, dtype=tf.float32), name='G')
        updates.append(G.assign_add(grad**2))
        with tf.control_dependencies(updates):
            updates.append(param.assign_sub(eta / tf.sqrt(G + eps) * grad))
    return updates

def rmsprop(cost, params, eta=0.001, rho=0.9, eps=1e-7):
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        G = tf.Variable(tf.zeros_like(param, dtype=tf.float32), name='G')
        updates.append(G.assign(rho * G + (1 - rho) * grad**2))
        with tf.control_dependencies(updates):
            updates.append(param.assign_sub(eta / tf.sqrt(G + eps) * grad))
    return updates

def compute_l2_reg(params): #追加
    l2_reg = 0
    for param in params:
        l2_reg += tf.reduce_sum(tf.square(param)) # 2 * tf.nn.l2_lossを使っても良い
    return l2_reg

def tf_log(x):
    # WRITE ME
    return tf.log(tf.clip_by_value(x, 1e-10, x))

##勝手に追加##
eta = 0.01 # 学習率
dropout_keep_prob = 0.5 # Dropout率
lmd = 0.001 # L2正則化項の係数
# batch_size = 32 # バッチサイズ
# n_epochs = 10 # epoch数

# WRITE ME
layers = [
    Dense(784, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 10, tf.nn.softmax)
]

# params = []
# h = x
# for layer in layers:
#     h = layer(h)
#     params += layer.params
# y = h
##上に代わってこちらを追加。内容は同等
def get_params(layers):
    params_all = []
    for layer in layers:
        params = layer.params
        params_all.extend(params)
    return params_all
def f_props(layers, h):
    for layer in layers:
        h = layer(h)
    return h

y = f_props(layers, x)
params_all = get_params(layers)

l2_reg = compute_l2_reg(params_all)

# WRITE ME
cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1)) + lmd * l2_reg

updates = adagrad(cost, params_all, eta) #適宜optimizerを変更
train = tf.group(*updates)

n_epochs = 100 #適宜変更 50くらいでいいかも
batch_size = 100
n_batches = math.ceil(len(x_train) / batch_size)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(n_epochs):
    # WRITE ME
    x_train, y_train = shuffle(x_train, y_train)
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        sess.run(train, feed_dict={x: x_train[start:end], t: y_train[start:end], is_training: True})
    y_pred, cost_valid_ = sess.run([y, cost], feed_dict={x: x_valid, t: y_valid, is_training: False})
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch + 1,
        cost_valid_,
        accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))

# WRITE ME
y_test = sess.run(y, feed_dict={x: x_test, is_training: False}) #sess.runで返すと、numpy配列が帰る
y_test = y_test.argmax(axis=1)
print(y_test)

submission = pd.Series(y_test, name='label')
submission.to_csv('/root/userspace/chap05/materials/submission_pred.csv', header=True, index_label='id')

Writing /root/userspace/chap05/materials/submission_code.py


In [58]:
print(y_valid.argmax(axis=1))
print(y_valid.shape)
print(y_pred.argmax(axis=1))
print(y_pred.shape)

[9 8 9 5 3 7 1 0 1 8 6 9 2 4 1 9 9 3 2 2 4 7 8 8 6 5 4 8 6 4 6 5 7 6 9 7 5
 7 7 8 4 3 0 4 2 4 5 0 1 1 4 6 9 2 8 5 6 8 8 8 5 2 4 4 3 4 8 6 6 9 0 8 4 0
 2 7 9 9 4 0 4 1 4 9 6 8 7 7 0 9 8 3 1 4 4 1 7 5 3 7 1 7 9 5 7 6 5 9 9 4 2
 3 5 0 6 9 2 4 5 3 5 0 2 9 7 0 9 9 1 4 4 6 8 5 1 4 4 1 8 9 4 2 2 0 2 6 4 6
 8 3 0 4 7 0 6 7 1 0 1 9 8 9 0 6 6 7 3 0 3 6 7 5 0 3 5 6 5 9 4 1 1 0 5 6 5
 3 4 7 5 2 4 2 3 1 7 5 4 3 5 6 3 3 7 5 2 4 0 8 8 5 6 8 8 9 3 3 7 7 1 3 7 1
 5 9 5 2 9 7 4 2 2 0 6 4 0 5 3 2 1 7 8 9 1 4 0 3 2 2 4 0 0 9 7 2 4 0 2 0 9
 0 2 7 1 7 8 4 2 3 1 7 7 0 7 4 7 1 8 9 7 2 0 4 3 6 5 9 5 8 2 1 4 0 7 4 8 3
 8 5 9 5 6 2 0 5 9 6 9 6 6 1 4 9 7 2 1 7 6 3 6 1 5 1 6 5 3 7 4 7 0 2 0 6 6
 7 6 8 8 4 6 1 6 0 9 7 3 3 0 9 1 3 5 8 3 1 5 1 6 3 2 9 1 0 9 9 8 9 5 1 7 8
 8 7 2 0 4 6 1 8 4 6 6 8 5 5 0 1 2 3 9 0 1 7 2 6 8 5 2 3 4 0 2 0 0 8 6 2 6
 3 2 7 7 0 0 7 6 6 9 4 6 5 8 6 1 4 3 3 0 6 0 8 0 3 4 7 5 2 9 2 8 7 0 3 4 9
 1 2 8 6 1 7 9 7 4 9 7 6 6 1 9 0 0 8 5 4 4 0 3 4 0 7 6 8 5 1 1 5 5 9 2 3 4
 8 2 6 4 6 3 1 1 8 9 0 5 

In [59]:
print(y_test)
print(y_test.shape)

[5 9 5 ... 8 9 0]
(10000,)
